# I. By Property Type

In [1]:
import pandas as pd
import os
import psycopg2

# To manage configurations
from decouple import config
from dotenv import load_dotenv

from sqlalchemy import create_engine


def compute_property_indices(env_path,year=2022):
    desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
    quarters = [f'{year}Q{i}' for i in range(1, 5)]
    
    load_dotenv(env_path)
    database_host = config("DATABASE_HOST")
    database_name = config("DATABASE_NAME")
    database_user = config("DATABASE_USER")
    database_password = config("DATABASE_PASSWORD")

    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host=database_host,
        dbname=database_name,
        user=database_user,
        password=database_password
    )

    # Create a cursor object to interact with the database
    cur = conn.cursor()
    # Create an SQLAlchemy engine for future database operations
    engine = create_engine(f"postgresql+psycopg2://{database_user}:{database_password}@{database_host}/{database_name}")

    # Function to fetch data
    def fetch_data(query):
        cur.execute(query)
        result = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(result, columns=columns)

    
    # Fetch data
    df_apart = fetch_data("SELECT * FROM apartment_for_sale")
    df_house = fetch_data("SELECT * FROM house_for_sale")

    
    # Apartments data processing
    #...........................
    #...........................
    for idx, price in enumerate(df_apart['price']):
        if price < 90000000:
            df_apart.at[idx, 'price'] = 90000000
        elif price > 480000000:
            df_apart.at[idx, 'price'] = 480000000
    df_apart['district'] = df_apart['district'].str.lower().str.strip()
    df_apart = df_apart.dropna(subset=['date'])
    df_apart['date'] = pd.to_datetime(df_apart['date'])
    df_apart['quarter'] = df_apart['date'].dt.quarter

    df_apart_2022 = df_apart[df_apart['district'].isin(desired_districts) & (df_apart['date'].dt.year == year)]
    median_apart_prices_by_quarter = df_apart_2022.groupby('quarter')['price'].median()
    print("-"*49, "\nMedian Apartment prices by quarter - 2022:\n",median_apart_prices_by_quarter,"\n" + "-"*49, "\n")
    apart_mean_median_price = median_apart_prices_by_quarter.mean()
    print("-"*49, "\n Apartments' Mean of Median Prices:", apart_mean_median_price, "\n" + "-"*49, "\n")
    apart_index_by_quarter = (median_apart_prices_by_quarter / apart_mean_median_price)*100
    print("-"*49, "\n Apartment Index Price for each quarter:\n", apart_index_by_quarter.to_string(float_format="%.4f"), "\n" + "-"*49, "\n")
    median_apart_price_2022 = df_apart_2022['price'].median()
    print("-"*49, "\n Median Price of Apartment for Sale in 2022:", round(median_apart_price_2022),"\n" + "-"*49, "\n")
    sum_prices_apart = df_apart_2022['price'].sum()
    print("-"*49, "\n Sum of prices of Apartments in Kigali City - 2022:", sum_prices_apart, "\n" + "-"*49, "\n")
    number_of_apartments = len(df_apart_2022)
    print("-"*49, "\n Number of Apartments for sale in 2022:", number_of_apartments,"\n" + "-"*49, "\n")
    
    # Houses data processing
    #.......................
    #.......................
    for idx, price in enumerate(df_house['price']):
        if price < 10000000:
            df_house.at[idx, 'price'] = 10000000
        elif price > 250000000:
            df_house.at[idx, 'price'] = 250000000
    df_house['district'] = df_house['district'].str.lower().str.strip()
    df_house = df_house.dropna(subset=['date'])
    df_house['date'] = pd.to_datetime(df_house['date'])
    df_house['quarter'] = df_house['date'].dt.quarter

    df_house_2022 = df_house[df_house['district'].isin(desired_districts) & (df_house['date'].dt.year == year)]
    median_house_prices_by_quarter = df_house_2022.groupby('quarter')['price'].median()
    print("-"*49, "\nMedian House prices by quarter - 2022:\n",median_house_prices_by_quarter,"\n" + "-"*49, "\n")
    house_mean_median_price = median_house_prices_by_quarter.mean()
    print("-"*49, "\n Houses' Mean of Median Prices:", house_mean_median_price, "\n" + "-"*49, "\n")
    house_index_by_quarter = (median_house_prices_by_quarter / house_mean_median_price)*100
    print("-"*49, "\n House Index Price for each quarter:\n", house_index_by_quarter.to_string(float_format="%.4f"), "\n" + "-"*49, "\n")
    median_house_price_2022 = df_house_2022['price'].median()
    print("-"*49, "\n Median Price of Houses for Sale in 2022:", round(median_house_price_2022),"\n" + "-"*49, "\n")
    sum_prices_house = df_house_2022['price'].sum()
    print("-"*49, "\n Sum of prices of Houses in Kigali City - 2022:", sum_prices_house, "\n" + "-"*49, "\n")
    number_of_houses = len(df_house_2022)
    print("-"*49, "\n Number of Houses for sale in 2022:", number_of_houses,"\n" + "-"*49, "\n")

    
    # Computing weight by property type
    #..................................
    Total_property_prices = sum_prices_house + sum_prices_apart
    print("-"*49, "\n Total Property price - 2022:", Total_property_prices, "\n" + "-"*49, "\n")
    weight_house = sum_prices_house / Total_property_prices
    print("-"*49, "\n House Weight - 2022:", weight_house, "\n" + "-"*49, "\n")
    weight_apart = sum_prices_apart / Total_property_prices
    print("-"*49, "\n Apartment Weight - 2022:", weight_apart, "\n" + "-"*49, "\n")

    
    # Computing aggregated index by property type
    # ...........................................
    # ...........................................
    prop_df_aggr_list = []
    for quarter_str in quarters:
        quarter_int = int(quarter_str[-1])
        if quarter_int in apart_index_by_quarter and quarter_int in house_index_by_quarter:
            property_aggregate_index = (apart_index_by_quarter[quarter_int] * weight_apart) + (house_index_by_quarter[quarter_int] * weight_house)
            prop_df_aggr_list.append((quarter_str, property_aggregate_index))
            continue
        else:
            print(f"No data available for {quarter_str}")

    
    # Print aggregated index values for each quarter
    print("-" * 49)
    print("Aggregated Index Price for each quarter:\n")
    for quarter, value in prop_df_aggr_list:
        print(f"{quarter:<10} | {value:.4f}")
    print("-" * 49 + "\n")
    
    # Creation of the aggregated DataFrame
    quarters = ['2022Q1', '2022Q2', '2022Q3', '2022Q4']
    prop_data_apart = list(zip(quarters, apart_index_by_quarter.values))
    prop_data_house = list(zip(quarters, house_index_by_quarter.values))
    apart_data_aggr = list(zip(quarters, [val for _, val in prop_df_aggr_list]))
    
    # Create dataframes for each dataset
    prop_df_house = pd.DataFrame(prop_data_house, columns=['Quarter', 'Houses'])
    prop_df_apart = pd.DataFrame(prop_data_apart, columns=['Quarter', 'Apartments'])
    prop_df_aggr = pd.DataFrame(apart_data_aggr, columns=['Quarter', 'Aggregated Index'])

    # Merge dataframes on 'Quarter' column
    prop_df_merged = pd.merge(prop_df_house, prop_df_apart, on='Quarter')
    prop_df_merged = pd.merge(prop_df_merged, prop_df_aggr, on='Quarter')
    prop_df_merged = prop_df_merged.round(2)


    # Save the merged DataFrame to a CSV file
    prop_df_merged.to_csv('Quarterly_Indices_2022_byPropertyType.csv', index=False)
    
    # Saving to DB:
    #.............
    table_name = 'Quarterly_Indices_2022_byPropertyType'
    prop_df_merged.to_sql(table_name, engine, if_exists='replace', index=False)
    print(f"Data saved to the table: {table_name}")

    print(prop_df_merged)

    # Close the cursor and connection
    cur.close()
    conn.close()

In [2]:
# Calling the function
env_path = os.path.abspath('.env')
compute_property_indices(env_path)

------------------------------------------------- 
Median Apartment prices by quarter - 2022:
 quarter
1    210000000.0
2    290000000.0
3     90000000.0
4     90000000.0
Name: price, dtype: float64 
------------------------------------------------- 

------------------------------------------------- 
 Apartments' Mean of Median Prices: 170000000.0 
------------------------------------------------- 

------------------------------------------------- 
 Apartment Index Price for each quarter:
 quarter
1   123.5294
2   170.5882
3    52.9412
4    52.9412 
------------------------------------------------- 

------------------------------------------------- 
 Median Price of Apartment for Sale in 2022: 290000000 
------------------------------------------------- 

------------------------------------------------- 
 Sum of prices of Apartments in Kigali City - 2022: 13835000000 
------------------------------------------------- 

------------------------------------------------- 
 Number of A

# II. By region

In [173]:
import pandas as pd

def process_and_analyze_data(env_path, year=2022):
    desired_districts = ['gasabo', 'kicukiro', 'nyarugenge']
    load_dotenv(env_path)
    database_host = config("DATABASE_HOST")
    database_name = config("DATABASE_NAME")
    database_user = config("DATABASE_USER")
    database_password = config("DATABASE_PASSWORD")

    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        host=database_host,
        dbname=database_name,
        user=database_user,
        password=database_password
    )

    # Create a cursor object to interact with the database
    cur = conn.cursor()
    # Create an SQLAlchemy engine for future database operations
    engine = create_engine(f"postgresql+psycopg2://{database_user}:{database_password}@{database_host}/{database_name}")

    # Function to fetch data
    def fetch_data(query):
        cur.execute(query)
        result = cur.fetchall()
        columns = [desc[0] for desc in cur.description]
        return pd.DataFrame(result, columns=columns)
    

    
    # Fetch data
    df_apart = fetch_data("SELECT * FROM apartment_for_sale")
    df_house = fetch_data("SELECT * FROM house_for_sale")


    # Pre-processing function for both apartment and house data
    def pre_process_data(df, type_data=''):
        # Price adjustment
        if type_data == 'apartment':
            df['price'] = df['price'].apply(lambda x: 90000000 if x < 90000000 else (480000000 if x > 480000000 else x))
        elif type_data == 'house':
            df['price'] = df['price'].apply(lambda x: 10000000 if x < 10000000 else (250000000 if x > 250000000 else x))

        # Convert the district names to lowercase and remove any leading or trailing spaces
        df['district'] = df['district'].str.lower().str.strip()

        # Drop any rows where the date is NaN
        df.dropna(subset=['date'], inplace=True)

        # Convert the date column to a datetime format
        df['date'] = pd.to_datetime(df['date'])

        # Extract and create a quarter column based on the date
        df['quarter'] = df['date'].dt.quarter

        # Filter the dataframe based on the desired districts and year
        return df[df['district'].isin(desired_districts) & (df['date'].dt.year == year)]

    df_apart_filtered = pre_process_data(df_apart, 'apartment')
    df_house_filtered = pre_process_data(df_house, 'house')

    # Analysis function for a given dataset (either apartment or house)
    def compute_statistics_and_analyze(df, type_data):
        for district_name in desired_districts:
            district_data = df[df['district'] == district_name]
            num_apart = len(district_data)
            print("-" * 49, f"\n Number of {type_data.capitalize()}s in {district_name.capitalize()} district - {year}:", num_apart,
                "\n" + "-" * 49, "\n")
            quarters_median_prices = district_data.groupby('quarter')['price'].median()
            quarters_without_houses = set([1, 2, 3, 4]) - set(quarters_median_prices.index)
            for quarter in quarters_without_houses:
                quarters_median_prices.loc[quarter] = 0
            quarters_median_prices.sort_index(inplace=True)

            median_price = district_data["price"].median()
            mean_median_price = quarters_median_prices.mean()
            index_by_quarter = (quarters_median_prices / mean_median_price) * 100

            print("-" * 49, f"\n {year} Median prices for each quarter in {district_name.capitalize()}:", quarters_median_prices, 
                "\n" + "-" * 49, "\n")
            print("-" * 49, f"\n {type_data.capitalize()} Median Prices in {district_name.capitalize()}:", median_price,
                "\n" + "-" * 49, "\n")
            print("-" * 49, f"\n Mean of Median Prices in {district_name.capitalize()}:", mean_median_price,
                "\n" + "-" * 49, "\n")
            print("-" * 49, f"\n Index for each quarter in {district_name.capitalize()}:", index_by_quarter.round(2).astype(str).str.rstrip('0').str.rstrip('.'),
                "\n" + "-" * 49, "\n")

    compute_statistics_and_analyze(df_apart_filtered, 'apartment')
    compute_statistics_and_analyze(df_house_filtered, 'house')

    ## Total price for base period (2022) - Apartments
    sum_prices_apart = df_apart_filtered.groupby('district')['price'].sum()

    ## District weight - Apartments
    apart_weights = sum_prices_apart / sum_prices_apart.sum()

    ## Total price for base period (2022) - Houses
    sum_prices_house = df_house_filtered.groupby('district')['price'].sum()

    ## District weight - Houses
    house_weights = sum_prices_house / sum_prices_house.sum()

    # Create DataFrame for property distribution and save it as CSV
    locations = ['gasabo', 'kicukiro', 'nyarugenge']
    num_apartments = [df_apart_filtered[df_apart_filtered['district'] == district].shape[0] for district in desired_districts]
    num_houses = [df_house_filtered[df_house_filtered['district'] == district].shape[0] for district in desired_districts]

    data = {
        'Location': locations,
        'Apartments': num_apartments,
        'Houses': num_houses,
        'Apartment Weights': [apart_weights[district] for district in desired_districts],
        'House Weights': [house_weights[district] for district in desired_districts]
    }

    df_distribution = pd.DataFrame(data)
    df_distribution.to_csv('CountsAndWeights_byRegion.csv', index=False)
    
    #Saving to DB:
    #............
    counts_table_name = 'CountsAndWeights_byRegion'
    df_distribution.to_sql(counts_table_name, engine, if_exists='replace', index=False)
    print(f"Data saved to the table: {counts_table_name}")
    
    # New functionality to create CSVs for indices
    def generate_indices_csv(df_apart, df_house):
        apart_indices_data = {"Quarter": [f"{year}Q{i}" for i in range(1, 5)]}
        house_indices_data = {"Quarter": [f"{year}Q{i}" for i in range(1, 5)]}

        # Calculate the district weights
        apart_weights = df_apart.groupby('district')['price'].sum() / df_apart['price'].sum()
        house_weights = df_house.groupby('district')['price'].sum() / df_house['price'].sum()

        def fill_indices_data(df, type_data):
            for district_name in desired_districts:
                district_data = df[df['district'] == district_name]
                quarters_median_prices = district_data.groupby('quarter')['price'].median()
                mean_median_price = quarters_median_prices.mean()
                index_by_quarter = ((quarters_median_prices / mean_median_price) * 100).round(2)

                indices_list = []
                for i in range(1, 5):  # For each quarter
                    indices_list.append(index_by_quarter.get(i, 0))

                if type_data == 'apartment':
                        apart_indices_data[district_name.capitalize()] = indices_list
                else:
                    house_indices_data[district_name.capitalize()] = indices_list

            # Calculate and append aggregate index
            aggregate_indices = []
            for i in range(1, 5):
                aggregate_index = sum([apart_indices_data[district_name.capitalize()][i-1] * apart_weights[district_name]
                                       for district_name in desired_districts] if type_data == 'apartment'
                                      else [house_indices_data[district_name.capitalize()][i-1] * house_weights[district_name]
                                                for district_name in desired_districts])
                aggregate_indices.append(round(aggregate_index, 2))

            if type_data == 'apartment':
                apart_indices_data["Aggregated Index"] = aggregate_indices
            else:
                house_indices_data["Aggregated Index"] = aggregate_indices

        fill_indices_data(df_apart, 'apartment')
        fill_indices_data(df_house, 'house')

        df_indices_apart = pd.DataFrame(apart_indices_data)
        df_indices_house = pd.DataFrame(house_indices_data)
        df_indices_apart.to_csv('Apartment_Quarterly_Indices_2022_byRegion.csv', index=False)
        df_indices_house.to_csv('House_Quarterly_Indices_2022_byRegion.csv', index=False)

        # Saving to DB:
        #.............
        apart_table_name = 'Apartment_Quarterly_Indices_2022_byRegion'
        df_indices_apart.to_sql(apart_table_name, engine, if_exists='replace', index=False)
        print(f"Data saved to the table: {apart_table_name}")
        
        house_table_name = 'House_Quarterly_Indices_2022_byRegion'
        df_indices_house.to_sql(house_table_name, engine, if_exists='replace', index=False)
        print(f"Data saved to the table: {house_table_name}")
    
        print("Apartment indices CSV generated successfully!")
        print(df_indices_apart)
        print("House indices CSV generated successfully!")
        print(df_indices_house)

    generate_indices_csv(df_apart_filtered, df_house_filtered)


    # Close the cursor and connection
    cur.close()
    conn.close()

In [174]:
# Calling the function
env_path = os.path.abspath('.env')
process_and_analyze_data(env_path)

------------------------------------------------- 
 Number of Apartments in Gasabo district - 2022: 33 
------------------------------------------------- 

------------------------------------------------- 
 2022 Median prices for each quarter in Gasabo: quarter
1    130000000.0
2    450000000.0
3     90000000.0
4     90000000.0
Name: price, dtype: float64 
------------------------------------------------- 

------------------------------------------------- 
 Apartment Median Prices in Gasabo: 170000000.0 
------------------------------------------------- 

------------------------------------------------- 
 Mean of Median Prices in Gasabo: 190000000.0 
------------------------------------------------- 

------------------------------------------------- 
 Index for each quarter in Gasabo: quarter
1     68.42
2    236.84
3     47.37
4     47.37
Name: price, dtype: object 
------------------------------------------------- 

------------------------------------------------- 
 Number of Ap